Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 6.2 ms, sys: 0 ns, total: 6.2 ms
Wall time: 11.3 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.141/10267/1 Dashboard: http://10.20.0.141:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 271 ms, sys: 53.8 ms, total: 324 ms
Wall time: 1.79 s


[1.6601529553154104,
 1.1347180501269811,
 1.472141427816501,
 1.4931291519268755,
 1.8315037567747914,
 1.7554713791700918,
 1.653061905165974,
 1.6388281708993655,
 1.7270214677009985,
 1.0955822428416262,
 1.755767061000427,
 1.5673743261158886,
 1.0776020716379153,
 1.4529845218073945,
 1.805809688418071,
 1.1687547512797125,
 1.721514063264379,
 1.9410345261174153,
 1.990735287849065,
 1.7742467495682956,
 1.7530867113446398,
 1.7313875746122895,
 1.8826920941678842,
 1.1603412186474036,
 1.6887372246605783,
 1.923568801418924,
 1.2002908885575543,
 1.558435668858326,
 1.0169493560899339,
 1.7336771608986172,
 1.9544681831255142,
 1.5295797672072218,
 1.0936865095157424,
 1.2614619643538356,
 1.3998678528165596,
 1.4067183980944429,
 1.498967367530407,
 1.3362021909682422]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.7428254735824096,
 1.9033567625024315,
 1.7639211180485765,
 1.8117933385765816,
 1.167642246741421,
 1.2552374160297892,
 1.2150042611087613,
 1.2154396821360658,
 1.478839466496796,
 1.5353995078164082,
 1.4015342080860145,
 1.109112560556568,
 1.7983803599951242,
 1.0246467210367185,
 1.2233174799186246,
 1.2078974574598256,
 1.5554821676771855,
 1.4345795778276287,
 1.717263220655564,
 1.2940972671399424,
 1.539505572525659,
 1.4443105558935274,
 1.8742820827576954,
 1.1803115901011463,
 1.3366858127712653,
 1.297995319969384,
 1.2141249152815883,
 1.2916566818575475,
 1.7146869520834427,
 1.8183762285227725,
 1.2265223928868698,
 1.0251825528970038,
 1.1701124451426559,
 1.7611741955121047,
 1.5809215840462927,
 1.6889790709124788,
 1.9857100620650576,
 1.8620975161540185,
 1.4461195178369246,
 1.9801577958331296,
 1.613372967625038,
 1.068491339266588,
 1.295174725871621,
 1.8969486316244,
 1.534595449590106,
 1.4137768225165899,
 1.228255629864649,
 1.837558459213252,
 1.3072

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 340 ms, sys: 34 ms, total: 374 ms
Wall time: 2.15 s
